In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
import string
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
import pandas as pd
import seaborn as sns
sns.set_style('darkgrid')
import warnings
warnings.filterwarnings('ignore')
import re
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack
from sklearn.multiclass import OneVsRestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
import PyPDF2
import csv
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText


job_description=input()




In [ ]:
# Function to extract text from a PDF file
def extract_text_from_pdf(pdf_path):
    t = ""
    with open(pdf_path, 'rb') as pdf_file:
        pdf_reader = PyPDF2.PdfFileReader(pdf_file)
        for page_num in range(pdf_reader.numPages):
            page = pdf_reader.getPage(page_num)
            t += page.extractText()
    return t

# Function to convert text to CSV
def convert_text_to_csv(text, csv_file):
    with open(csv_file, 'w', newline='') as csv_file:
        writer = csv.writer(csv_file)
        for line in text.split('\n'):
            writer.writerow([line])

def extract_text_from(path):
  return extract_text(path)

def extract_contact_number_from_resume(text):
  contact_number = None

    # Use regex pattern to find a potential contact number
  pattern = r"\b(?:\+?\d{1,3}[-.\s]?)?\(?\d{3}\)?[-.\s]?\d{3}[-.\s]?\d{4}\b"
  match = re.search(pattern, text)
  if match:
      contact_number = match.group()

  return contact_number

def extract_email_from_resume(text):
  email = None

    # Use regex pattern to find a potential email address
  pattern = r"\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}\b"
  match = re.search(pattern, text)
  if match:
    email = match.group()

  return email

def extract_skills_from_resume(text, skills_list):
  skills = []

  for skill in skills_list:
    pattern = r"\b{}\b".format(re.escape(skill))
    match = re.search(pattern, text, re.IGNORECASE)
    if match:
      skills.append(skill)

  return skills

def extract_education_from_resume(text):
  education = []

 # List of education keywords to match against
  for keyword in word:
    pattern = r"(?i)\b{}\b".format(re.escape(keyword))
    match = re.search(pattern, text)
    if match:
          education.append(match.group())
          return education

def extract_name(resume_text):
  name_regex = r"[A-Z][a-z]+\s[A-Z][a-z]+"
  name = re.search(name_regex, resume_text).group()
  return name


  if __name__ == '__main__':
    extract_text_from_pdf('er/resume.pdf')
    UpdatedResumeDataSet=convert_text_to_csv(t,'er/ResumeDataSet.csv')#Example Resume
    contact=''
    for resume_path in resume_paths:
        resume += extract_text_from_pdf(resume_path)

        print("Resume:", resume_path)

        name = extract_name(resume)
        if name:
            print("Name:", name)
        else:
            print("Name not found")

        contact_number = extract_contact_number_from_resume(text)
        if contact_number:
            print("Contact Number:", contact_number)
        else:
            print("Contact Number not found")

        email = extract_email_from_resume(text)
        if email:
            print("Email:", email)
        else:
            print("Email not found")

        skills_list = []
        extracted_skills = extract_skills_from_resume(text, skills_list)
        if extracted_skills:
            print("Skills:", extracted_skills)
        else:
            print("No skills found")

        extracted_education = extract_education_from_resume(text)
        if extracted_education:
            print("Education:", extracted_education)
        else:
            print("No education information found")

        print()



In [ ]:
def Candiate_CV(file):
  dff = pd.read_csv('UpdatedResumeDataSet.csv')
  dff['cleaned'] = dff['Resume'].apply(lambda x:Clean_file(x))
  #getting the entire resume text
  corpus=" "
  for i in range(0,len(dff)):
      corpus= corpus+ dff["cleaned"][i]

df=Candiate_CV('UpdatedResumeDataSet.csv')

In [ ]:
def Clean_file(text):
    # Remove punctuation and whitespace
    resumeText = re.sub('http\S+\s*', ' ', resumeText)  # remove URLs
    resumeText = re.sub('RT|cc', ' ', resumeText)  # remove RT and cc
    resumeText = re.sub('#\S+', '', resumeText)  # remove hashtags
    resumeText = re.sub('@\S+', '  ', resumeText)  # remove mentions
    resumeText = re.sub('[%s]' % re.escape("""!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~"""), ' ', resumeText)  # remove punctuations
    resumeText = re.sub(r'[^\x00-\x7f]',r' ', resumeText)
    resumeText = re.sub('\s+', ' ', resumeText)  # remove extra whitespace

    # Convert text to lowercase
    text = text.lower()

    return text.split()

def encode_text(tokens):
    # Create a vocabulary of unique tokens
    vocabulary = set(tokens)

    # Create a dictionary mapping each token to a unique integer index
    token_to_index = {token: i for i, token in enumerate(vocabulary)}

    # Encode the tokens into numerical representations
    encoded_tokens = [token_to_index[token] for token in tokens]

    return encoded_tokens


In [ ]:
tokenizer = nltk.tokenize.RegexpTokenizer('\w+')
#Tokenizing the text
tokens = tokenizer.tokenize(corpus)
len(tokens)
words = []
# Looping through the tokens and make them lower case
for word in tokens:
    words.append(word.lower())
words[0:5]
label = LabelEncoder()
df['new_Category'] = label.fit_transform(df['Category'])
df.head()

# Vectorizing the cleaned columns
text = df['cleaned'].values
target = df['new_Category'].values
word_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    stop_words='english',
    max_features=1500)
word_vectorizer.fit(text)
WordFeatures = word_vectorizer.transform(text)



In [ ]:
# Separate train and test data
X_train, X_test, y_train, y_test = train_test_split(WordFeatures, target, random_state=24, test_size=0.2)
# Model Training
model = OneVsRestClassifier(KNeighborsClassifier())
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

In [ ]:
#Stop words are generally the most common words in a language.
#English stop words from nltk.
stopwords = nltk.corpus.stopwords.words('english')
words_new = []
#Now we need to remove the stop words from the words variable
#Appending to words_new all words that are in words but not in sw
for word in words:
    if word not in stopwords:
        words_new.append(word)

# Lemmatization
wn = WordNetLemmatizer()
lem_words=[]
for word in words_new:
    word=wn.lemmatize(word)
    lem_words.append(word)

same=0
diff=0
for i in range(0,1832):
    if(lem_words[i]==words_new[i]):
        same=same+1
    elif(lem_words[i]!=words_new[i]):
        diff=diff+1



In [ ]:
def score_job_description(job_description, corpus):
    # Preprocess the job description and resume
    job_description_tokens = preprocess_text(job_description)
    resume_tokens = preprocess_text(corpus)

    # Convert the job description and resume tokens to numerical representations
    job_description_embedding = encode_text(job_description_tokens)
    resume_embedding = encode_text(resume_tokens)

    # Calculate the cosine similarity between the job description and resume embeddings
    similarity = tf.keras.backend.dot(job_description_embedding, resume_embedding)

    # Scale the similarity score to a range of 0 to 1
    score_22 = tf.math.sigmoid(similarity)

    return score_22.numpy()
score_2= score_job_description(job_description, corpus)

# Load the Universal Sentence Encoder model
module_url = "https://tfhub.dev/google/universal-sentence-encoder/4"
embed = hub.load(module_url)

# Define a function to compute the cosine similarity between two vectors
def cosine_similarity(vector_a, vector_b):
    dot_product = np.dot(vector_a, vector_b)
    norm_a = np.linalg.norm(vector_a)
    norm_b = np.linalg.norm(vector_b)
    similarity = dot_product / (norm_a * norm_b)
    return similarity

# Define a function to score job description with respect to a resume
def score_job_description_with_resume(job_description, corpus):
    # Encode job description and resume using the USE model
    job_description_vec = embed([job_description])[0]
    resume_vec = embed([corpus])[0]

    # Compute the cosine similarity between the two vectors
    similarity_score = cosine_similarity(job_description_vec, text)

    return similarity_score

score = score_job_description_with_resume(job_description, corpus)

result=int((0.63*score)+(0.37*score_2)*100)
if result > 80:
  # Create the email message
  msg = MIMEMultipart()
  msg['From'] = sender_email
  msg['To'] = email
  msg['Subject'] = subject
  msg.attach(MIMEText(message_1, 'plain'))
  try:
      server = smtplib.SMTP('smtp.gmail.com', 587)  # Use the SMTP server and port for your email provider
      server.starttls()  # Enable TLS encryption
      server.login(sender_email, sender_password)  # Login to your email account

      # Send the email
      server.sendmail(sender_email, receiver_email, msg.as_string())

      # Close the connection to the SMTP server
      server.quit()

      print("Email sent successfully to", receiver_email)
  except Exception as e:
      print("Error sending email:", str(e))
else:
    # Create the email message
    msg = MIMEMultipart()
    msg['From'] = sender_email
    msg['To'] = email
    msg['Subject'] = subject
    msg.attach(MIMEText(message_2, 'plain'))
    try:
        server = smtplib.SMTP('smtp.gmail.com', 587)  # Use the SMTP server and port for your email provider
        server.starttls()  # Enable TLS encryption
        server.login(sender_email, sender_password)  # Login to your email account

        # Send the email
        server.sendmail(sender_email, receiver_email, msg.as_string())

        # Close the connection to the SMTP server
        server.quit()

        print("Email sent successfully to", receiver_email)
    except Exception as e:
        print("Error sending email:", str(e))





In [ ]:
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

# Email configuration
sender_email = 'your_email@gmail.com'  # Your Gmail email address
sender_password = 'your_password'  # Your Gmail password
  # The recipient's email address
subject = 'Your Job Application Status'
message_1 = 'Congratulations!You are selected'
message_2 = 'Sorry,You are not selected'

# Create the email message
msg = MIMEMultipart()
msg['From'] = sender_email
msg['To'] = email
msg['Subject'] = subject
msg.attach(MIMEText(message, 'plain'))

# Establish a connection to the SMTP server (Gmail in this example)
try:
    server = smtplib.SMTP('smtp.gmail.com', 587)  # Use the SMTP server and port for your email provider
    server.starttls()  # Enable TLS encryption
    server.login(sender_email, sender_password)  # Login to your email account

    # Send the email
    server.sendmail(sender_email, receiver_email, msg.as_string())

    # Close the connection to the SMTP server
    server.quit()

    print("Email sent successfully to", receiver_email)
except Exception as e:
    print("Error sending email:", str(e))
